# Módulo MACHINE LEARNING
___

## Importamos librerías

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Adecuamos dataset de volúmen de tránsito

In [2]:
# Leer el archivo CSV en un DataFrame de pandas
df_tvc = pd.read_csv('4. Traffic_Volume_Counts_v4_FINAL.csv')

In [11]:
# Selecciona las columnas a partir de "Date" en adelante y excluye "Latitude" y "Longitude"
columnas_a_derretir = df_tvc.iloc[:, 6:-3]

# Derretir las columnas en filas
df_tvc = pd.melt(df_tvc, id_vars=['Date', 'location_id'], 
                    value_vars=columnas_a_derretir.columns, 
                    var_name='Hour', value_name='Volume')


In [12]:
# Crea un diccionario para mapear los nombres de las columnas actuales a los nombres deseados
nuevos_nombres = {
    '12:00-1:00 AM': 0,
    '1:00-2:00AM': 1,
    '2:00-3:00AM': 2,
    '3:00-4:00AM': 3,
    '4:00-5:00AM': 4,
    '5:00-6:00AM': 5,
    '6:00-7:00AM': 6,
    '7:00-8:00AM': 7,
    '8:00-9:00AM': 8,
    '9:00-10:00AM': 9,
    '10:00-11:00AM': 10,
    '11:00-12:00PM': 11,
    '12:00-1:00PM': 12,
    '1:00-2:00PM': 13,
    '2:00-3:00PM': 14,
    '3:00-4:00PM': 15,
    '4:00-5:00PM': 16,
    '5:00-6:00PM': 17,
    '6:00-7:00PM': 18,
    '7:00-8:00PM': 19,
    '8:00-9:00PM': 20,
    '9:00-10:00PM': 21,
    '10:00-11:00PM': 22,
    '11:00-12:00AM': 23
}

# Cambiar los nombres de las columnas 'Hour' según el diccionario
df_tvc['Hour'].replace(nuevos_nombres, inplace=True)

In [13]:
# Filtrar los registros donde 'location_id' es igual a 'No encontrado'
df_tvc = df_tvc.loc[df_tvc['location_id'] != 'No encontrado']

# Convertir la columna 'Date' a tipo datetime
df_tvc['Date'] = pd.to_datetime(df_tvc['Date'], errors='coerce')
df_tvc.loc[:, 'Date'] = pd.to_datetime(df_tvc['Date'], errors='coerce')

# Extraer los componentes de fecha
df_tvc.loc[:, 'day_week'] = df_tvc['Date'].dt.dayofweek

# Crear la nueva columna con la combinación de 'day', 'day_week', 'Hour' y 'location_id'
df_tvc.loc[:, 'combined_column'] = df_tvc['day_week'].astype(str) + '-' + df_tvc['Hour'].astype(str) + '-' + df_tvc['location_id'].astype(str)


## Adecuamos dataset de taxis

In [25]:
# Leer el archivo CSV en un DataFrame de pandas
df_yellow = pd.read_csv('yellowpreliminar.csv')

In [27]:
# Eliminar la primera columna
df_yellow.drop(columns=['Unnamed: 0'], inplace=True)

# Renombrar las columnas
df_yellow.rename(columns={'pickup_datetime': 'Date',
                          'Year': 'year',
                          'Month': 'month',
                          'Day': 'day',
                          'DayofWeek': 'day_week',
                          'PULocationID': 'location_id'}, inplace=True)

# Eliminar la columna 'pickup_borough'
df_yellow.drop(columns=['pickup_borough'], inplace=True)

In [28]:
# Crear la nueva columna con la combinación de 'day', 'day_week', 'Hour' y 'location_id'
df_yellow['combined_column'] = df_yellow['day_week'].astype(str) + '-' + df_yellow['Hour'].astype(str) + '-' + df_yellow['location_id'].astype(str)

# Realizar un agrupamiento y sumar la cantidad de registros
df_new = df_yellow.groupby('combined_column').size().reset_index(name='count')

## Unificamos los datasets

In [30]:
# Realizar un merge entre df_vol y df_new utilizando combined_column como clave
df_merged = df_tvc.merge(df_new, on='combined_column', how='left')

In [31]:
# Filtrar los NaN de df_merged
df_filtered = df_merged.dropna()

# Filtrar df_ml para eliminar registros con volume igual a 0
df_filtered = df_filtered[df_filtered['Volume'] > 0]

### Se realiza cálculo de optimización

In [33]:
# Calcula la optimización de volume vs count
df_filtered['optimization'] = df_filtered['count'] / (np.log(df_filtered['Volume'] + 1)+1)

In [35]:
df_ml = df_filtered.drop(columns=['Date', 'combined_column'])

# Definir las franjas horarias
def get_hour_franja(hour):
    if 0 <= hour < 6:
        return 1  # Franja horaria 1
    elif 6 <= hour < 12:
        return 2  # Franja horaria 2
    elif 12 <= hour < 18:
        return 3  # Franja horaria 3
    else:
        return 4  # Franja horaria 4

# Aplicar la función a df_ml para crear una nueva columna 'Hour_franja'
df_ml['Hour_franja'] = df_ml['Hour'].apply(get_hour_franja)

# Ahora puedes utilizar 'Hour_franja' en lugar de 'Hour' para entrenar el modelo
X = df_ml[['day_week', 'Hour_franja', 'location_id']]
y = df_ml['optimization']

## Función Entrenamiento

In [39]:
def train_random_forest(df_ml):
    # Seleccionar características de entrada (X) y variable objetivo (y)
    X = df_ml[['day_week', 'Hour', 'location_id']]
    y = df_ml['optimization']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Inicializar el modelo de Random Forests
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)

    # Evaluar el modelo con los datos de prueba
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Mostrar las métricas de evaluación
    print(f"Error cuadrático medio (MSE): {mse}")
    print(f"Coeficiente de determinación (R^2): {r2}")

    return model

## Función de Uso

In [41]:
def get_top_5_location_id(date, hour, model, df_ml):
    # Convertir la fecha a day_week
    date = pd.to_datetime(date)
    day_week = date.dayofweek

    # Crear un DataFrame con todas las combinaciones de location_id, day_week, y Hour
    unique_locations = df_ml['location_id'].unique()
    test_data = pd.DataFrame({
        'location_id': unique_locations,
        'day_week': [day_week] * len(unique_locations),
        'Hour': [hour] * len(unique_locations)
    })

    test_data = test_data[['day_week', 'Hour', 'location_id']]

    # Predecir el valor de optimization para todas las combinaciones
    predictions = model.predict(test_data)

    # Añadir las predicciones al DataFrame
    test_data['optimization'] = predictions

    # Ordenar el DataFrame por la columna optimization en orden descendente
    test_data_sorted = test_data.sort_values(by='optimization', ascending=False)

    # Seleccionar los 5 mejores location_id
    top_5_locations = test_data_sorted.head(5)['location_id']

    # Retornar los 5 mejores location_id
    return list(top_5_locations)

## Entrenamos modelo

In [43]:
# Entrenar el modelo de Random Forests con df_ml
model = train_random_forest(df_ml)

Error cuadrático medio (MSE): 12.152064629818092
Coeficiente de determinación (R^2): 0.983214951379792


## Ejemplo de uso

In [51]:
date = "2024-01-01"  # Fecha de ejemplo
hour = 5  # Hora de ejemplo

top_5_locations = get_top_5_location_id(date, hour, model, df_ml)
print("Top 5 location_id:", top_5_locations)

Top 5 location_id: ['132', '48', '68', '230', '79']
